# Data Engineering

In [390]:
import pandas as pd
import numpy as np
import re

import fundamentus

# from google.cloud import storage
# import os

from utils import initialize_bucket
import requests
import requests_cache
import logging
import time
from datetime import date

In [391]:
credentials_path = 'datascience-capstone-project-05b1642f45c3.json'

## 1 - Data Collection

For the data gathering, two repositories on Github were combined, so that the historical data could be scraped from the web. This Raw data is beeing stored in the cloud (GCP) and are the main data used for the project.

Repositories utilized:
* https://github.com/mv/fundamentus-api
* https://github.com/Victorcorcos/bovespa-winner

In [392]:
def perc_to_float(val):
    """
    Percent to float
      - replace string in pt-br to float
      - from '45,56%' to 0.4556
    Input:
        (DataFrame, column_name)
    """

    res = val
    res = res.replace( to_replace=r'[%]', value='' , regex=True )
    res = res.replace( to_replace=r'[.]', value='' , regex=True )
    res = res.replace( to_replace=r'[,]', value='.', regex=True )
    res = res.astype(float) / 100

    return res

def _rename_cols(data):
    """
    Rename columns in DataFrame
      - use a valid Python identifier
      - so each column can be a DataFrame property
      - Example:
          df.pl > 0
    """

    df2 = pd.DataFrame()

    ## Fix: rename columns
    df2['cotacao'  ] = data['Cotação'          ]
    df2['pl'       ] = data['P/L'              ]
    df2['pvp'      ] = data['P/VP'             ]
    df2['psr'      ] = data['PSR'              ]
    df2['dy'       ] = data['Div.Yield'        ]
    df2['pa'       ] = data['P/Ativo'          ]
    df2['pcg'      ] = data['P/Cap.Giro'       ]
    df2['pebit'    ] = data['P/EBIT'           ]
    df2['pacl'     ] = data['P/Ativ Circ.Liq'  ]
    df2['evebit'   ] = data['EV/EBIT'          ]
    # df2['evebitda' ] = data['EV/EBITDA'        ]
    df2['mrgebit'  ] = data['Mrg Ebit'         ]
    df2['mrgliq'   ] = data['Mrg. Líq.'        ]
    df2['roic'     ] = data['ROIC'             ]
    df2['roe'      ] = data['ROE'              ]
    df2['liqc'     ] = data['Liq. Corr.'       ]
    df2['liq2m'    ] = data['Liq.2meses'       ]
    df2['patrliq'  ] = data['Patrim. Líq'      ]
    df2['divbpatr' ] = data['Dív.Brut/ Patrim.']
    df2['c5y'      ] = data['Cresc. Rec.5a'    ]

    return df2

def get_resultado_raw(url):
    """
    Get data from fundamentus:
      URL:
        http://fundamentus.com.br/resultado.php
    RAW:
      DataFrame preserves original HTML header names
    Output:
      DataFrame
    """

    ##
    ## Busca avançada por empresa
    ##
    # url = 'http://www.fundamentus.com.br/resultado.php'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01',
           'Accept-Encoding': 'gzip, deflate',
           }

    with requests_cache.enabled():
        content = requests.get(url, headers=hdr)

        if content.from_cache:
            logging.debug('.../resultado.php: [CACHED]')
        else: # pragma: no cover
            logging.debug('.../resultado.php: sleeping...')
            time.sleep(.500) # 500 ms


    ## parse + load
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]

    ## Fix: percent string
    df['Div.Yield']     = perc_to_float( df['Div.Yield']     )
    df['Mrg Ebit']      = perc_to_float( df['Mrg Ebit']      )
    df['Mrg. Líq.']     = perc_to_float( df['Mrg. Líq.']     )
    df['ROIC']          = perc_to_float( df['ROIC']          )
    df['ROE']           = perc_to_float( df['ROE']           )
    df['Cresc. Rec.5a'] = perc_to_float( df['Cresc. Rec.5a'] )

    ## index by 'Papel', instead of 'int'
    df.index = df['Papel']
    df.drop('Papel', axis='columns', inplace=True)
    df.sort_index(inplace=True)

    ## naming
    df.name = 'Fundamentus: HTML names'
    df.columns.name = 'Multiples'
    df.index.name = 'papel'

    ## return sorted by 'papel'
    return df


def get_resultado(url):
    """
    Data from fundamentus, fixing header names.
      URL:
        given from the user
      Obs:
        DataFrame uses short header names
    Output:
      DataFrame
    """

    ## get RAW data
    data1 = get_resultado_raw(url)

    ## rename!
    data2 = _rename_cols(data1)

    ## metadata
    data2.name = 'Fundamentus: short names'
    data2.columns.name = 'Multiples'
    data2.index.name = 'papel'

    ## remove duplicates
#   df = data2.drop_duplicates(subset=['cotacao','pl','pvp'], keep='last')
    df = data2.drop_duplicates(keep='first')

    return df


In [393]:


# def initialize_bucket(credentials_path, create_bucket=False):

#     os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

#     client = storage.Client()
#     bucket = client.bucket('storage-barsianize')
    
#     if create_bucket:
#         bucket.location = 'US-EAST1'
#         bucket.create()

#     return client, bucket
    

In [394]:
def get_windows(max, min, window=5, step=1):

    maxes = list(range(min+window,max+1, step))
    mins = list(range(min, max+1-window, step))
    sequences = []
    if len(maxes)==len(mins):
        for i in range(len(maxes)):
            sequence = list(range(mins[i],maxes[i]+1,step))
            sequences.append(sequence)
        return sequences
    else:
        print(f'error!!! min {maxes}, max {maxes}')

### Get historical data

In [395]:
urls = {
    2008: 'https://web.archive.org/web/20080613050801/http://www.fundamentus.com.br/resultado.php',
    2009: 'https://web.archive.org/web/20090123022224/http://www.fundamentus.com.br/resultado.php',
    2010: 'https://web.archive.org/web/20100115191626/http://www.fundamentus.com.br/resultado.php',
    2011: 'https://web.archive.org/web/20110113192117/http://www.fundamentus.com.br/resultado.php',
    2012: 'https://web.archive.org/web/20120106023830/http://www.fundamentus.com.br/resultado.php',
    2013: 'https://web.archive.org/web/20130105004012/http://www.fundamentus.com.br/resultado.php',
    2014: 'https://web.archive.org/web/20140108164618/http://www.fundamentus.com.br/resultado.php',
    2015: 'https://web.archive.org/web/20150119231047/http://www.fundamentus.com.br/resultado.php',
    2016: 'https://web.archive.org/web/20160106101916/http://www.fundamentus.com.br/resultado.php',
    2017: 'https://web.archive.org/web/20170505164235/http://www.fundamentus.com.br/resultado.php',
    2018: 'https://web.archive.org/web/20180105120409/http://www.fundamentus.com.br/resultado.php',
    2019: 'https://web.archive.org/web/20190102202956/http://www.fundamentus.com.br/resultado.php',
    2020: 'https://web.archive.org/web/20200122200313/http://www.fundamentus.com.br/resultado.php',
    2021: 'https://web.archive.org/web/20210227034423/http://www.fundamentus.com.br/resultado.php',
    2022: 'https://web.archive.org/web/20220314021607/http://www.fundamentus.com.br/resultado.php',
    2023: 'https://web.archive.org/web/20230324145930/http://www.fundamentus.com.br/resultado.php'
  }
  
years = list(range(2008,2024))

def get_dates(urls):
    dict_dates = {}
    for key, value in urls.items():
        str_1 = re.split('/web/',value)[1]
        str_2 = re.split('/http',str_1)[0]
        str_date = f'{str_2[:4]}-{str_2[4:6]}-{str_2[6:8]}'
        dict_dates[int(str_2[:4])] = pd.to_datetime(str_date)
    return dict_dates

dates = get_dates(urls)

df_full = pd.DataFrame()
for year in years:
    df = get_resultado(urls[year])
    df['year'] = year
    df['date'] = dates[year]
    df_full = pd.concat([df_full,df])
    
df_full = df_full.reset_index()

In [396]:
df_full.head()

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,...,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
0,ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.00,0.0,1.107010e+09,0.00,0.0000,2008,2008-06-13
1,ABCB4,9.00,0.00,1.10,0.000,0.0489,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.00,4160460.0,1.107010e+09,0.00,0.0000,2008,2008-06-13
2,ABNB3,17.35,12.39,3.27,1.845,0.0254,2.590,7.26,8.45,5.04,...,0.1490,0.3465,0.2639,3.28,2586090.0,2.733490e+08,0.00,0.1446,2008,2008-06-13
3,ABYA3,16.00,25.67,4.13,4.399,0.0079,0.570,2.10,17.45,2.51,...,0.1714,0.0356,0.1608,1.94,5915230.0,1.971610e+08,2.40,0.0000,2008,2008-06-13
4,ACES3,95.27,8.79,2.28,1.693,0.0000,1.438,6.40,7.20,3.77,...,0.1927,0.2690,0.2595,2.05,181013.0,3.105800e+09,0.09,0.1334,2008,2008-06-13


### Get detailed information about each ticker

In [397]:
papeis = df_full['papel'].unique()

df_papeis = pd.DataFrame()
for papel in papeis:
    try:
        df = fundamentus.get_papel(papel)
        df_papeis = pd.concat([df_papeis,df])
    except:
        print(f'fail papel {papel}')

2023-04-02 12:53:19,205 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,269 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,329 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,350 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,413 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,476 [detalhes.get_papel] INFO: detalhes: call: get..._papel()


fail papel ABNB3


2023-04-02 12:53:19,544 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,608 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,672 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,735 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,797 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,859 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,925 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:19,944 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,008 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,072 [detalhes.get_papel] INFO: detalhes: call: get..._papel()


fail papel AGEN11


2023-04-02 12:53:20,137 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,202 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,266 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,330 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,394 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,463 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,529 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,594 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,659 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,724 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,789 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53:20,853 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:53

...

2023-04-02 12:54:10,035 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,100 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,163 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,225 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,289 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,352 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,362 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,426 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,489 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,552 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,616 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,680 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54

fail papel OGSA3


2023-04-02 12:54:10,932 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:10,996 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,060 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,124 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,188 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,251 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,313 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,376 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,439 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,502 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,565 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:11,629 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54

fail papel LIQO3


2023-04-02 12:54:13,929 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:13,992 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,054 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,117 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,179 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,190 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,253 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,316 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,379 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,444 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,508 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54:14,570 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-04-02 12:54

In [398]:
df_full[df_full['papel']=='ITUB4']

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,...,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
1885,ITUB4,37.91,19.86,3.55,0.0,0.0188,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1786,0.0,1.686560e+08,4.886160e+10,0.0,0.1767,2010,2010-01-15
2654,ITUB4,40.25,14.55,3.22,0.0,0.0246,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2210,0.0,2.156390e+08,5.722510e+10,0.0,0.2734,2011,2011-01-13
3440,ITUB4,34.66,13.27,2.09,0.0,0.0292,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1572,0.0,2.463310e+08,7.591600e+10,0.0,-0.4082,2012,2012-01-06
4236,ITUB4,35.51,14.42,1.93,0.0,0.0325,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1337,0.0,2.728300e+08,8.416040e+10,0.0,-0.4905,2013,2013-01-05
5051,ITUB4,31.27,14.83,1.80,0.0,0.0316,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1214,0.0,3.127270e+08,8.733480e+10,0.0,-0.4519,2014,2014-01-08
5870,ITUB4,33.10,12.07,1.88,0.0,0.0299,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1559,0.0,4.247690e+08,9.726990e+10,0.0,-0.1391,2015,2015-01-19
6689,ITUB4,25.34,6.96,1.43,0.0,0.0584,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2050,0.0,4.617880e+08,1.080200e+11,0.0,0.2599,2016,2016-01-06
7524,ITUB4,38.10,12.31,2.13,0.0,0.0468,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1733,0.0,4.294830e+08,1.175570e+11,0.0,0.1019,2017,2017-05-05
8372,ITUB4,45.10,13.70,2.35,0.0,0.0334,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1717,0.0,4.841310e+08,1.255750e+11,0.0,0.0391,2018,2018-01-05
9224,ITUB4,35.50,17.59,2.77,0.0,0.0595,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1576,0.0,7.078580e+08,1.255340e+11,0.0,-0.1029,2019,2019-01-02


In [399]:
df_papeis.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_med_2m,...,Rec_Servicos_3m,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,Div_Liquida,Receita_Liquida_12m,EBIT_12m,Receita_Liquida_3m,EBIT_3m
ABCB3,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,0.00,1899-12-30,0.00,0.00,0,...,118020000,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABCB4,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,17.91,2023-01-04,14.06,22.30,13880200,...,118020000,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABYA3,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,4.91,2010-02-11,0.00,0.00,0,...,NaN,-766000,29556000,498222000,383159000,353603000,250657000,25802000,67990000,18057000
ACES3,ACES3,ON,ARCELORMITTAL INOX BRASIL ON,Siderurgia e Metalurgia,Siderurgia,95.27,2008-04-18,0.00,0.00,0,...,NaN,163789000,922471000,2157470000,279166000,-643305000,4182420000,983634000,949114000,174503000
ACES4,ACES4,PN,ARCELORMITTAL INOX BRASIL PN,Siderurgia e Metalurgia,Siderurgia,94.73,2008-04-24,0.00,0.00,0,...,NaN,163789000,922471000,2157470000,279166000,-643305000,4182420000,983634000,949114000,174503000


In [400]:
df_full.head()

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,...,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
0,ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.00,0.0,1.107010e+09,0.00,0.0000,2008,2008-06-13
1,ABCB4,9.00,0.00,1.10,0.000,0.0489,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.00,4160460.0,1.107010e+09,0.00,0.0000,2008,2008-06-13
2,ABNB3,17.35,12.39,3.27,1.845,0.0254,2.590,7.26,8.45,5.04,...,0.1490,0.3465,0.2639,3.28,2586090.0,2.733490e+08,0.00,0.1446,2008,2008-06-13
3,ABYA3,16.00,25.67,4.13,4.399,0.0079,0.570,2.10,17.45,2.51,...,0.1714,0.0356,0.1608,1.94,5915230.0,1.971610e+08,2.40,0.0000,2008,2008-06-13
4,ACES3,95.27,8.79,2.28,1.693,0.0000,1.438,6.40,7.20,3.77,...,0.1927,0.2690,0.2595,2.05,181013.0,3.105800e+09,0.09,0.1334,2008,2008-06-13


### Storing the raw data into Google Cloud

In [401]:
client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('raw/df_full.csv')
blob.upload_from_string(df_full.to_csv(), 'text/csv')
# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('raw/df_papeis.csv')
blob.upload_from_string(df_papeis.to_csv(), 'text/csv')
# upload raw fundamentalist data to Google Cloud Storage
# blob = bucket.blob('raw/categories.csv')
# blob.upload_from_string(categories.to_csv(), 'text/csv')

## 2 - Data Cleansing

### Load data

In [402]:
client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

path = "gs://storage-barsianize/raw/df_full.csv"
df_full =  pd.read_csv(path, index_col=0)

path = "gs://storage-barsianize/raw/df_papeis.csv"
df_papeis = pd.read_csv(path, index_col=0)

Desired information about each ticker

In [403]:
df_papeis.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_med_2m,...,Rec_Servicos_3m,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,Div_Liquida,Receita_Liquida_12m,EBIT_12m,Receita_Liquida_3m,EBIT_3m
ABCB3,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,0.00,1899-12-30,0.00,0.0,0,...,118020000.0,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABCB4,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,17.91,2023-01-04,14.06,22.3,13880200,...,118020000.0,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABYA3,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,4.91,2010-02-11,0.00,0.0,0,...,NaN,-766000,29556000.0,4.982220e+08,383159000.0,353603000.0,2.506570e+08,25802000.0,67990000.0,18057000.0
ACES3,ACES3,ON,ARCELORMITTAL INOX BRASIL ON,Siderurgia e Metalurgia,Siderurgia,95.27,2008-04-18,0.00,0.0,0,...,NaN,163789000,922471000.0,2.157470e+09,279166000.0,-643305000.0,4.182420e+09,983634000.0,949114000.0,174503000.0
ACES4,ACES4,PN,ARCELORMITTAL INOX BRASIL PN,Siderurgia e Metalurgia,Siderurgia,94.73,2008-04-24,0.00,0.0,0,...,NaN,163789000,922471000.0,2.157470e+09,279166000.0,-643305000.0,4.182420e+09,983634000.0,949114000.0,174503000.0


In [404]:
df_papeis.columns

Index(['Papel', 'Tipo', 'Empresa', 'Setor', 'Subsetor', 'Cotacao',
       'Data_ult_cot', 'Min_52_sem', 'Max_52_sem', 'Vol_med_2m',
       'Valor_de_mercado', 'Valor_da_firma', 'Ult_balanco_processado',
       'Nro_Acoes', 'PL', 'PVP', 'PEBIT', 'PSR', 'PAtivos', 'PCap_Giro',
       'PAtiv_Circ_Liq', 'Div_Yield', 'EV_EBITDA', 'EV_EBIT', 'Cres_Rec_5a',
       'LPA', 'VPA', 'Marg_Bruta', 'Marg_EBIT', 'Marg_Liquida', 'EBIT_Ativo',
       'ROIC', 'ROE', 'Liquidez_Corr', 'Div_Br_Patrim', 'Giro_Ativos', 'Ativo',
       'Cart_de_Credito', 'Depositos', 'Patrim_Liq', 'Result_Int_Financ_12m',
       'Rec_Servicos_12m', 'Lucro_Liquido_12m', 'Result_Int_Financ_3m',
       'Rec_Servicos_3m', 'Lucro_Liquido_3m', 'Disponibilidades',
       'Ativo_Circulante', 'Div_Bruta', 'Div_Liquida', 'Receita_Liquida_12m',
       'EBIT_12m', 'Receita_Liquida_3m', 'EBIT_3m'],
      dtype='object')

In [405]:
# info_papeis = ['Papel','Tipo', 'Empresa', 'Setor', 'Subsetor','Data_ult_cot']
info_papeis = ['Papel','Tipo', 'Empresa', 'Setor', 'Subsetor','Data_ult_cot']
               
df_papeis_clean = df_papeis[info_papeis]

df_papeis_clean = df_papeis_clean.dropna()
df_papeis_clean = df_papeis_clean.drop_duplicates()

In [406]:
df_papeis_clean['Data_ult_cot'] = pd.to_datetime(df_papeis_clean['Data_ult_cot'])

In [407]:
df_completed = df_papeis_clean.merge(df_full, how='left', left_index=True, right_on='papel').drop_duplicates()

In [408]:
df_completed.isna().mean().sort_values(ascending=False)

Papel           0.0
Tipo            0.0
year            0.0
c5y             0.0
divbpatr        0.0
patrliq         0.0
liq2m           0.0
liqc            0.0
roe             0.0
roic            0.0
mrgliq          0.0
mrgebit         0.0
evebit          0.0
pacl            0.0
pebit           0.0
pcg             0.0
pa              0.0
dy              0.0
psr             0.0
pvp             0.0
pl              0.0
cotacao         0.0
papel           0.0
Data_ult_cot    0.0
Subsetor        0.0
Setor           0.0
Empresa         0.0
date            0.0
dtype: float64

In [409]:
client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('trusted/df_completed.csv')
blob.upload_from_string(df_completed.to_csv(), 'text/csv',)

## 3 - Data Preparation

In [410]:
client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

In [411]:
path = "gs://storage-barsianize/trusted/df_completed.csv"
df_completed =  pd.read_csv(path, index_col=0)
df_completed['Data_ult_cot'] = pd.to_datetime(df_completed['Data_ult_cot'])

In [412]:
df_completed.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Data_ult_cot,papel,cotacao,pl,pvp,...,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
0,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,1.107010e+09,0.0,0.0000,2008,2008-06-13
736,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.1553,0.0,0.0,1.158760e+09,0.0,0.0000,2009,2009-01-23
1478,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.1016,0.0,0.0,1.182180e+09,0.0,-0.1233,2010,2010-01-15
2237,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.1521,0.0,0.0,1.312080e+09,0.0,0.1043,2011,2011-01-13
3020,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.1571,0.0,0.0,1.460150e+09,0.0,0.1272,2012,2012-01-06


In [413]:
df_completed.columns

Index(['Papel', 'Tipo', 'Empresa', 'Setor', 'Subsetor', 'Data_ult_cot',
       'papel', 'cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg', 'pebit',
       'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc', 'liq2m',
       'patrliq', 'divbpatr', 'c5y', 'year', 'date'],
      dtype='object')

In [414]:
df_completed['Data_ult_cot_year'] = df_completed['Data_ult_cot'].dt.year

In [415]:
df_actual = df_completed[df_completed['Data_ult_cot_year'] == pd.to_datetime(date.today()).year]

In [416]:
tickers = df_actual['Papel'].unique()

In [417]:
tickers

array(['ABCB4', 'AGRO3', 'ALPA3', 'ALPA4', 'AZEV3', 'AZEV4', 'BAHI3',
       'BALM3', 'BALM4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BDLL4',
       'BEEF3', 'BEES3', 'BGIP4', 'BIOM3', 'BMEB3', 'BMEB4', 'BMIN3',
       'BMIN4', 'BMKS3', 'BOBR4', 'BRAP3', 'BRAP4', 'BRGE11', 'BRGE3',
       'BRIV3', 'BRIV4', 'BRKM3', 'BRKM5', 'BRML3', 'BRSR3', 'BRSR5',
       'BRSR6', 'CALI3', 'CAMB3', 'CBEE3', 'CCRO3', 'CEBR3', 'CEBR5',
       'CEBR6', 'CEDO4', 'CEEB3', 'CGAS3', 'CGAS5', 'CGRA3', 'CGRA4',
       'CLSC3', 'CMIG3', 'CMIG4', 'COCE5', 'CPFE3', 'CPLE3', 'CPLE6',
       'CRIV3', 'CRIV4', 'CSAB3', 'CSAB4', 'CSAN3', 'CSMG3', 'CSNA3',
       'CSRN3', 'CSRN5', 'CSRN6', 'CTKA4', 'CTNM4', 'CTSA3', 'CTSA4',
       'CYRE3', 'DASA3', 'DOHL4', 'EALT3', 'EALT4', 'EKTR3', 'EKTR4',
       'ELET3', 'ELET6', 'EMAE4', 'EMBR3', 'ENBR3', 'ENGI3', 'ENGI4',
       'EQTL3', 'ESTR4', 'ETER3', 'EUCA3', 'EUCA4', 'EVEN3', 'EZTC3',
       'FESA3', 'FESA4', 'FHER3', 'GEPA3', 'GEPA4', 'GFSA3', 'GGBR3',
       'GGBR4', 'GO

In [418]:
stock_info = ['Papel','Tipo','Empresa','Setor','Subsetor']

indicators = ['cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg', 'pebit',
              'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc', 'liq2m',
              'patrliq', 'divbpatr', 'c5y','date','year']

In [419]:
df_indicators = df_actual[np.isin(df_actual['Papel'],tickers)][stock_info + indicators]

In [420]:
data = df_actual[df_actual['Papel']==tickers[2]].sort_values('year', ascending=False)

max = data['year'].max()
min = data['year'].min()
windows = get_windows(max, min, window=5, step=1)

### Get relative information for each sector

In [421]:
rel_base = df_actual.groupby('Setor').agg({
                    'mrgliq':'mean',
                    'liq2m':'mean',
                    'patrliq':'mean',
                    'mrgebit':'mean',
                    'c5y':'mean'
                })

In [422]:
cols_to_rel = ['mrgliq','liq2m','patrliq','mrgebit','c5y']

def relativize(data, ref_data, cols, sector_col):
    """
    Relativize the values in a DataFrame by dividing them by the corresponding values in a reference DataFrame.

    Parameters
    ----------
    data : pandas.DataFrame
        The DataFrame to relativize.
    ref_data : pandas.DataFrame
        The reference DataFrame containing the values to divide `data` by.
    cols : list of str
        The column names to relativize.
    sector_col : str
        The name of the column containing the sector information.

    Returns
    -------
    pandas.DataFrame
        A copy of the input DataFrame `data` with the specified columns relativized.
    """
    data_out = data.copy()
    sectors = data_out[sector_col].unique()
    
    for sector in sectors:
        sector_data = data_out.loc[data_out[sector_col] == sector]
        for col in cols:
            sector_data[col] /= ref_data.at[sector, col]
        data_out.loc[data_out[sector_col] == sector] = sector_data
    
    return data_out

In [423]:
base_dataset = relativize(df_actual, rel_base, cols_to_rel, 'Setor')

C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\1672380360.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_data[col] /= ref_data.at[sector, col]


In [424]:
# columns_to_drop = ['Lucro_Liquido_3m',\
#                'Disponibilidades','Ativo_Circulante','Div_Bruta','Div_Liquida','Receita_Liquida_12m',\
#                'EBIT_12m','Receita_Liquida_3m','EBIT_3m']

# base_dataset.drop(columns_to_drop, axis=1, inplace=True)

In [425]:
def get_variations(data, windows, stock_info_cols):
    """
    Compute various statistics for a set of rolling windows of data.

    Parameters
    ----------
    data : pandas.DataFrame
        The input DataFrame containing the data to compute statistics for.
    windows : list of list of int
        A list of rolling windows to compute statistics for. Each window should be a list of years.
    stock_info_cols : list of str
        The names of the columns containing stock information.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing the computed statistics for each rolling window.
    """
    
    data_line_full = pd.DataFrame()
    cols = ['cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg',
            'pebit', 'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc',
            'liq2m', 'patrliq', 'divbpatr', 'c5y']
    extra_cols = list(data.columns[~np.isin(data.columns, stock_info_cols + cols)])
    
    for window in windows:
        window.sort(reverse=True)

        data_window = data[np.isin(data['year'],window)]
        data_window = data_window.sort_values(by='year', ascending=False)

        if np.sum(data['year']>np.max(window)):

            data_line = {}

            #get label
            data_line['dy_label'] = data[data['year'] == (np.max(window)+1)]['dy'].values[0]

            for col in cols:
                data_line[col] = data_window[col].iloc[0]
                try:
                    data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
                except:
                    data_line[f'{col}_rel'] = 0

                data_line[f'{col}_median'] = data_window[col].median()

                data_line[f'{col}_mean'] = data_window[col].mean()

                try:
                    data_line[f'{col}_mean_median_ratio'] = data_line[f'{col}_mean']/data_line[f'{col}_median']
                except:
                    data_line[f'{col}_rel'] = 0

                data_line[f'{col}_std'] = data_window[col].std()

                try:
                    data_line[f'{col}_spread'] = (data_window[col].max() - data_window[col].min()) / data_window[col].mean()
                except:
                    data_line[f'{col}_spread'] = 0

                try:
                    data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
                except:
                    data_line[f'{col}_spread_rel_now'] = 0

            for extra_col in extra_cols:
                data_line[extra_col] = data_window[extra_col].iloc[0]


            data_line_full = pd.concat([data_line_full, pd.DataFrame(data_line, index=[0])])
        
        else:
            continue
    
    statistic_cols = list(data_line_full.columns)
    extra_cols = list(data.columns[~np.isin(data.columns, stock_info_cols + cols)])

    data_line_full[stock_info_cols] = data[stock_info].iloc[0]
    data_line_full = data_line_full[stock_info_cols + statistic_cols]

    return data_line_full
        


In [426]:
# get_variations(data, windows, stock_info)

In [427]:
def get_full_variations(data, tickers, window, stock_info_cols):

    df_variations = pd.DataFrame()

    for ticker in tickers:

        data_ticker = data[data['Papel']==ticker].sort_values('year', ascending=False)

        max = data_ticker['year'].max()
        min = data_ticker['year'].min()
        windows = get_windows(max, min, window=window, step=1)

        print(ticker)

        ticker_variation = get_variations(data_ticker, windows, stock_info_cols)

        df_variations = pd.concat([df_variations, ticker_variation])

    return df_variations


In [428]:
base_dataset = get_full_variations(base_dataset, tickers, window=5, stock_info_cols=stock_info)

ABCB4
AGRO3


C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_

ALPA3
ALPA4


C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{c

AZEV3


...

ENMT4
MEAL3
PRIO3
AALR3
AFLT3
ALUP4
ATOM3


C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{co

CRPG5
CRPG6
EGIE3
ENGI11
FRTA3
MOVI3
PARD3
RAIL3


C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:63: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_13452\3187901427.py:42: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}

RNEW3
RNEW4
STBP3
AZUL4
BPAC5
CAML3
CRFB3
DMMO3
IRBR3
SUZB3
WIZS3
WLMM4
B3SA3
BPAC11
HAPV3
SAPR11
ALSO3
APER3
BMGB4
CEAB3
COGN3
ENAT3
LOGG3
NEOE3
NTCO3
SQIA3
TAEE3
TAEE4
TASA3
TASA4
VIVA3
YDUQ3
AERI3
ALPK3
AMBP3
ATMP3
AURA33
AVLL3
BOAS3
CASH3
CURY3
DMVF3
ENJU3
EQMA3B
EQPA3
EQPA5
ESPA3
GMAT3
GPAR3
HBRE3
HBSA3
INTB3
LAVV3
LJQQ3
LWSA3
MBLY3
MDNE3
MELK3
MTRE3
NGRD3
PDTC3
PETZ3
PGMN3
PLPL3
PRNR3
RDOR3
RRRP3
SEQL3
SIMH3
SOMA3
TFCO4
TIMS3
VAMO3
AESB3
AGXY3
ALLD3
AMER3
ARML3
ASAI3
BLAU3
BMOB3
BRBI11
BRIT3
CBAV3
CLSA3
CMIN3
CPLE11
CSED3
CXSE3
DESK3
DEXP3
DEXP4
DOTZ3
DXCO3
ELMD3
EPAR3
FIQE3
GGPS3
IFCM3
IGTI11
IGTI3
JALL3
KRSA3
LVTC3
MATD3
MEGA3
MLAS3
MODL3
NINJ3
ONCO3
OPCT3
ORVR3
PORT3
RAIZ4
RCSL3
RECV3
SBFG3
SMFT3
SOJA3
SYNE3
TRAD3
TTEN3
VBBR3
VIIA3
VITT3
VVEO3
WEST3
AURE3
CSUD3
NEXP3
VSTE3
WIZC3
ZAMP3


In [429]:
base_dataset.columns[:100]

Index(['Papel', 'Tipo', 'Empresa', 'Setor', 'Subsetor', 'dy_label', 'cotacao',
       'cotacao_rel', 'cotacao_median', 'cotacao_mean',
       'cotacao_mean_median_ratio', 'cotacao_std', 'cotacao_spread',
       'cotacao_spread_rel_now', 'pl', 'pl_rel', 'pl_median', 'pl_mean',
       'pl_mean_median_ratio', 'pl_std', 'pl_spread', 'pl_spread_rel_now',
       'pvp', 'pvp_rel', 'pvp_median', 'pvp_mean', 'pvp_mean_median_ratio',
       'pvp_std', 'pvp_spread', 'pvp_spread_rel_now', 'psr', 'psr_rel',
       'psr_median', 'psr_mean', 'psr_std', 'psr_spread', 'psr_spread_rel_now',
       'dy', 'dy_rel', 'dy_median', 'dy_mean', 'dy_mean_median_ratio',
       'dy_std', 'dy_spread', 'dy_spread_rel_now', 'pa', 'pa_rel', 'pa_median',
       'pa_mean', 'pa_std', 'pa_spread', 'pa_spread_rel_now', 'pcg', 'pcg_rel',
       'pcg_median', 'pcg_mean', 'pcg_std', 'pcg_spread', 'pcg_spread_rel_now',
       'pebit', 'pebit_rel', 'pebit_median', 'pebit_mean', 'pebit_std',
       'pebit_spread', 'pebit_spread_

In [438]:
columns_to_drop = ['cotacao','cotacao_median', 'cotacao_mean','c5y_std']
base_dataset.drop(columns_to_drop,axis=1, inplace=True)

In [439]:
def inpute_median_by_category(data,group_by_col):

    cols_with_nans = data.columns[data.isna().any()].tolist()

    for col in cols_with_nans:
       data[col] = data.groupby(group_by_col)[col].transform(lambda x: x.fillna(x.median()))

    return data


In [440]:
inpute_median_by_category(base_dataset,'Setor').isna().sum().sort_values(ascending=False)

Papel                         0
liqc_mean                     0
roe_mean_median_ratio         0
roe_std                       0
roe_spread                    0
                             ..
pcg_spread                    0
pcg_spread_rel_now            0
pebit                         0
pebit_rel                     0
divbpatr_mean_median_ratio    0
Length: 159, dtype: int64

In [441]:
base_dataset = inpute_median_by_category(base_dataset,'Setor')
# base_dataset.fillna(np.median, inplace=True)
base_dataset.isna().sum().sort_values(ascending=False)[:20]



Papel                      0
liqc_mean                  0
roe_mean_median_ratio      0
roe_std                    0
roe_spread                 0
roe_spread_rel_now         0
liqc                       0
liqc_rel                   0
liqc_median                0
liqc_std                   0
roe_median                 0
liqc_spread                0
liqc_spread_rel_now        0
liq2m                      0
liq2m_rel                  0
liq2m_median               0
liq2m_mean                 0
liq2m_mean_median_ratio    0
roe_mean                   0
roe_rel                    0
dtype: int64

In [442]:
# base_dataset = base_dataset.drop(columns_to_drop, axis=1)
base_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
base_dataset = base_dataset.dropna()
base_dataset.isna().sum().sort_values(ascending=False)[:20]

Papel                      0
liqc_mean                  0
roe_mean_median_ratio      0
roe_std                    0
roe_spread                 0
roe_spread_rel_now         0
liqc                       0
liqc_rel                   0
liqc_median                0
liqc_std                   0
roe_median                 0
liqc_spread                0
liqc_spread_rel_now        0
liq2m                      0
liq2m_rel                  0
liq2m_median               0
liq2m_mean                 0
liq2m_mean_median_ratio    0
roe_mean                   0
roe_rel                    0
dtype: int64

### Storing refined data into Google Cloud

In [443]:
client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

# upload base dataset to Google Cloud Storage
blob = bucket.blob('refined/base_dataset.csv')
blob.upload_from_string(base_dataset.to_csv(), 'text/csv',)

> To Dos
* pegar primeiras 23 features e pensar em feature engineering
    * relativizar os valores absolutos (mrgliq, Lucro, patrliq, divbrut, ativo_circulante, receita_liquida, etc)
    * testar outras formas de scaling
    * spread relativo sempre ao now

